Software Engineering Challenge- DRAGON FRUIT

SAKETH REDDY NARAHARI

sakethre@usc.edu

Applied for Software Engineer/ Full Stack Engineer position


### Q1
Come up with efficient data structures to represent both types of images: those generated
by the microscope, and those generated by the dye sensor. These need not have the same
representation; the only requirement is that they be compact and take as little storage
space as possible. Explain why you picked the representation you did for each image type,
and if possible estimate how much storage would be taken by the images. What is the
worst-case storage size in bytes for each image representation you chose?

Q1. Ans : 

For the two types of images generated by the microscope and the dye sensor, the code employs different data structures for compact and efficient representation:

1. **Microscope Image (Generated by the Microorganism):**
   - **Data Structure Used:** Run Length Encoding (RLE)
   - **Explanation:** RLE is chosen because microscope images often have large continuous regions of either background or microorganisms. RLE represents consecutive identical pixels with a count, which is very effective for such images. This compression technique reduces the storage space required.
   - **Storage Estimate:** In the worst-case scenario, where the image has alternating values (010101...), each pixel would be represented as a sequence of (1, value). Assuming each pixel value (0/1) takes 1 byte and using 10 bits for counting, the worst-case size for one RLE pair is 10 bits + 8 bits = 18 bits (or 2.25 bytes). Therefore, the worst-case size for an `IMG_SIZE x IMG_SIZE` image is `IMG_SIZE * IMG_SIZE * 2.25 bytes`.

2. **Dye Sensor Image (Generated by the Dye Sensor):**
   - **Data Structure Used:** Sparse Matrix Representation or Dictionary Encoding
   - **Explanation:** Dye sensor images often have mostly empty regions with sporadic occurrences of dye (1s). Using a sparse matrix representation or dictionary encoding, only the coordinates of non-zero values (where dye is present) are stored. This is efficient when there are only a few regions with positive values.
   - **Storage Estimate:** For each `(x, y)` pair, we need to store two integer values. Assuming each integer is 4 bytes, the worst-case size for the entire image (where the entire image has dye cells) would be `2 * 4 bytes * IMG_SIZE * IMG_SIZE`.

These data structures are chosen to optimize storage for the specific characteristics of microscope and dye sensor images, providing efficient representation while minimizing storage space.

### Q2
Before the researchers give you real images to work with, you would like to test out any
code you write. To this end, you would like to create “fake” simulated images and pretend
they were captured by the microscope and the dye sensor. Using the data structures you
chose in (1) above, write code to create such simulated images. Try and be as realistic in the
generated images as possible.


Q2 Ans :

The functions `generate_microorganism_img` and `generate_cancer_img` are crucial for simulating images in response to Q2. 

1. **generate_microorganism_img:**
   - **Purpose:** This function creates a simulated image of a microorganism captured by the microscope. It uses the Graham's scan algorithm to generate a realistic blob-like shape, simulating the appearance of a microorganism under the microscope.
   - **Why:** This function allows testing and validation of the code by generating synthetic microscope images, helping ensure that the chosen data structure (Run Length Encoding - RLE) effectively represents the characteristics of microscope images.

2. **generate_cancer_img:**
   - **Purpose:** This function generates a simulated image of a microorganism with cancer cells, as captured by the dye sensor. It randomly places cancer cells within the microorganism blob, mimicking the presence of luminescent dye in certain regions.
   - **Why:** This function facilitates testing of the cancer detection algorithm. By creating synthetic images with known cancerous regions, it allows evaluating the accuracy of the code in identifying cancer cells. The generated image is then compressed and checked for the presence and extent of cancer, supporting the validation process.


In [4]:
INT_MAX = 100000
IMG_SIZE = 1000  # IMG_SIZE = 100000

# Function to create a simulated image of a microorganism captured by the microscope
def generate_microorganism_img(img_size=IMG_SIZE):
    print("A sample random image generation started...")

    # Find convex hull from randomly generated points by Gaussian distribution
    pts = convexHull([(random.gauss(img_size / 2, img_size / 5), random.gauss(img_size / 2, img_size / 5)) for _ in range(10)])

    # Connect all the points via smooth interpolation
    xs, ys = [interpolateSmoothly(zs, 10) for zs in zip(*pts)]
    blob_points = [list(a) for a in zip(xs, ys)]

    # Display the microorganism blob generated from above
    plt.plot(xs + [xs[0]], ys + [ys[0]])
    plt.xlim([0, img_size])
    plt.ylim([0, img_size])
    plt.show()

    positive_pixel_count = 0

    rows, cols = (img_size, img_size)
    arr = [[int(0) for _ in range(cols)] for _ in range(rows)]

    # Fill the array with 1s for pixels inside the microorganism blob
    for i in range(rows):
        for j in range(cols):
            if is_inside_polygon(points=blob_points, p=[j, i]):
                arr[i][j] = 1  # Point inside the enclosed region
                positive_pixel_count += 1

    end = time.time()
    print("%.3f seconds took to generate a random microorganism image of size %dx%d" % (end - start, img_size, img_size))
    print("\t%.2f percent of the image is occupied with the microorganism blob" % (positive_pixel_count / (img_size ** 2) * 100))

    return arr

# Function to create a simulated image of a microorganism with cancer cells captured by the dye sensor
def generate_cancer_img(img_size=IMG_SIZE):
    print("\nA sample cancer image generation started...")

    rows, cols = (img_size, img_size)
    arr = [[int(0) for _ in range(cols)] for _ in range(rows)]

    for _ in range(10):  # Try generating 10 cancer cells in the sample image
        rand_x = random.randrange(0, IMG_SIZE)
        rand_y = random.randrange(0, IMG_SIZE)

        pts = convexHull([(random.gauss(rand_x, img_size / 50), random.gauss(rand_y, img_size / 50)) for _ in range(5)])
        xs, ys = [interpolateSmoothly(zs, 10) for zs in zip(*pts)]

        cancer_cell_bounding_coordinates = [list(a) for a in zip(xs, ys)]

        for i in range(rows):
            for j in range(cols):
                if is_inside_polygon(points=cancer_cell_bounding_coordinates, p=[j, i]):
                    arr[i][j] = 1  # Point inside the enclosed region

    end = time.time()
    print("%.3f seconds took to generate a random cancer image of size %dx%d" % (end - start, img_size, img_size))

    return arr


# Display the image using matplotlib
def display_image(img, title):
    plt.title(title)
    plt.imshow(img, cmap='gray')
    plt.xlim([0, IMG_SIZE])
    plt.ylim([0, IMG_SIZE])
    plt.show()



### Q3
Using the simulated images generated by the code you wrote for (2) above as input, write a
function to compute whether a parasite has cancer or not.

Q3 Ans : 

The function `check_if_has_cancer` serves the purpose of evaluating the presence and extent of cancer in a simulated microorganism image, generated by the microscope and dye sensor simulations. By comparing the decompressed image of the microorganism (captured by the microscope) with the simulated image containing cancer cells (captured by the dye sensor), the function calculates the percentage of the microorganism's body that exhibits cancerous regions. This percentage is crucial for assessing the accuracy of the cancer detection algorithm, providing insights into its performance when applied to synthetic images. The function plays a pivotal role in validating the code's ability to correctly identify cancerous regions in microscopic images.



In [ ]:
# Function to determine the presence and extent of cancer in the micro-organism
def check_if_has_cancer(compressed_seqs, random_cancer_image):
    decompressed_img = image_decoding(compressed_seqs, (IMG_SIZE, IMG_SIZE))
    
    # Rename variables for convenience
    blob = decompressed_img
    cancer = random_cancer_image

    true_positive = 0
    true_negative = 0
    false_positive = 0
    false_negative = 0

    for i in range(IMG_SIZE):
        for j in range(IMG_SIZE):
            if blob[i][j] == 1:
                if cancer[i][j] == 1:
                    true_positive += 1
                else:
                    true_negative += 1
            else:
                if cancer[i][j] == 1:
                    false_positive += 1
                else:
                    true_negative += 1
    
    # Compute cancer region percentage relative to the entire organism body
    cancer_percent = true_positive / (true_positive + true_negative) * 100

    return cancer_percent



### Q4
You give your code from (3) to the researchers, who run it and find that it is running too
slowly for their liking. What can you do to improve the execution speed? Write the code to
implement the fastest possible version you can think of for the function in (3).


Q4 Ans : 

Since we are working with large amount of data and programming with it, to improve its speed , we can consider follwig options: 

1. Use various libraries such as numpy etc to find out if there are functions that'd help us to execute code faster
2. Eliminate redundancies, re calculations or nested loops using effecient programming methids such as Dynamic Programming
3. In lab setting, we can consider Cythonization:
Cython is a superset of Python designed to improve performance. By adding type annotations and compiling the code, Cython can generate C code, leading to faster execution.
4.  Parallelization:
Use parallel processing to distribute the workload across multiple cores, especially when dealing with large images.
5. Also we can think of trying hardware accleration/boost for faster executions.


In our code, we can make our check_if_cancer code optimized by leveraging NumPy's array operations and avoiding explicit nested loops. NumPy operations are optimized and significantly faster than traditional Python loops. Here's the optimized version of the function:

In [5]:
import numpy as np

def check_if_has_cancer_optimized(compressed_seqs, random_cancer_image):
    decompressed_img = image_decoding(compressed_seqs, (IMG_SIZE, IMG_SIZE))

    # Convert the images to NumPy arrays for faster computations
    blob = np.array(decompressed_img)
    cancer = np.array(random_cancer_image)

    # Calculate the true positive, true negative, false positive, and false negative
    true_positive = np.sum(np.logical_and(blob == 1, cancer == 1))
    true_negative = np.sum(np.logical_and(blob == 0, cancer == 0))
    false_positive = np.sum(np.logical_and(blob == 0, cancer == 1))
    false_negative = np.sum(np.logical_and(blob == 1, cancer == 0))

    # Compute cancer region percentage relative to the entire organism body
    cancer_percent = true_positive / (true_positive + true_negative) * 100

    return cancer_percent




### Q5
What other compression techniques can you suggest for both types of images (parasite and
dye)? How would they impact runtime? Can you compute actual runtime and storage costs
for typical images (not oversimplified image such as a circle for the parasite, or simple
straight lines or random points for dye) in your code? The measurements should be done on
your computer with an actual image size of 100,000x100,000 pixels (and not a scaled down
version).

q5 ans

For the microscope images (parasite), one alternative compression technique is **Lempel-Ziv-Welch (LZW) Compression**. LZW is a dictionary-based compression algorithm that works well for images with repetitive patterns, as it can identify and replace repeated sequences with shorter codes. The impact on runtime would depend on the nature of the image. For images with substantial redundancy, LZW could achieve good compression ratios, potentially reducing storage requirements. However, the runtime impact might be noticeable, as the compression process involves dictionary lookups and code assignments.

For the dye sensor images, since they contain sparse data, a suitable compression technique is **Run-Length Encoding (RLE)**, which is already implemented in the provided code. RLE works particularly well when consecutive pixels have the same value, efficiently representing long runs of identical data. The impact on runtime is generally low for RLE since it involves a straightforward process of counting and encoding consecutive runs.

Code integrate LZW compression for the microscope imagesis in next cell:


It's essential to note that the actual impact on runtime and storage costs would depend on the characteristics of the specific images. Highly complex and detailed images may benefit more from sophisticated compression techniques, while simpler images may not show significant improvements. Additionally, the computational cost of compression and decompression should be considered in the overall analysis.

For a comprehensive evaluation, you can use Python's `timeit` module to measure the execution time of compression and decompression operations for various images. Keep in mind that the execution time will vary based on the content and complexity of the images, and it's advisable to test with a representative set of images to draw meaningful conclusions about the performance of different compression techniques.

In [ ]:
import zlib

def image_lzw_compress(img):

    flattened_img = np.ravel(img)
    img_bytes = bytes(flattened_img)
    compressed_data = zlib.compress(img_bytes, level=zlib.Z_BEST_COMPRESSION)
    return compressed_data

def image_lzw_decompress(compressed_data, shape):
    decompressed_data = zlib.decompress(compressed_data)
    decompressed_img = np.frombuffer(decompressed_data, dtype=np.uint8)
    return decompressed_img.reshape(shape)

### Q6
Describes what tools you used to solve the challenge, particularly any LLM techniques.

Q6 Ans : 
LLMs used : ChatGPT- with GPT 3.5,  Bing search with GPT 4, BardAI
Tools used :Google, Various Github repositories